In [ ]:
# set up deps

#python3 -m venv .venv
#source .venv/bin/activate 
#docker compose down -v && docker compose up && python3 ./app/populate_db.py
import dotenv
import os
import sys
from google import genai
dotenv.load_dotenv(".env", override=True)
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])
sys.path.insert(0, os.path.abspath('app'))


In [2]:
domain_expert_prompt_template = """You are an expert in Xenon short-arc discharge lamps made by OSRAM, typically used in cinema projectors, searchlights, and industrial applications.
 Give some hints on how to select the appropriate lighting or lightings for the customer's use case. We have the following information about the lightings: {json_schema}
 Your advice will be used in constructing a database query. Try to keep it short and prefer using numeric fields. Do not suggest using the content field.
 "application_areas" have the following possible values:
[
    "Digitale Film- und Videoprojektion",
    "Sonnensimulation",
    "Architektur- und Effektlicht („Light Finger“)",
    "Klassische 35-mm-Filmprojektion",
]

""product_family_benefits"" have the following possible values:
[
    "Hohe Lichtbogenstabilität",
    "Konstante Farbtemperatur von 6.000 K über die gesamte Lebensdauer der Lampe",
    "Einfach zu warten",
    "Breiter Dimmbereich",
    "Sofort Licht auf der Leinwand dank Heißwiederzündungsfunktion",
    "Kurzbogen mit sehr hoher Leuchtdichte für hellere Leinwandausleuchtung",
]


The customer question is: {input}
"""

sql_gen_prompt = """You are a PostgreSQL expert.
Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Use the following advice from a lighting expert to compose the query that best matches the advice. : {expert_advice}

When user asks for recommendation, create derived columns for text fields and order the result set based on these in order to provide results ordered by relevance.
When user asks for recommendation, it is forbidden to use WHERE clauses as we should not filter out everything accidentally. Instead create derived column and order by it. Be mindful of the order of the order commands. 
Only use WHERE clause, when the user supplies exact parameters for searching products.
Do not make up new text fields.


Given you need to retrieve an item with min or max aggregation, make sure to return all items with min or max value. Same rule applies when multiple items fulfil the condition.
Make sure to take text[] fields into consideration - do not treat them like simple text fields. 
You must filter them like: SELECT * FROM xenon_lamps WHERE ANY(application_areas) LIKE '%Filmprojektion%';

Use the following information for recommendations when suitable:

"application_areas" have the following possible values:
[
    "Digitale Film- und Videoprojektion",
    "Sonnensimulation",
    "Architektur- und Effektlicht („Light Finger“)",
    "Klassische 35-mm-Filmprojektion",
]

"product_family_benefits" have the following possible values:
[
    "Hohe Lichtbogenstabilität",
    "Konstante Farbtemperatur von 6.000 K über die gesamte Lebensdauer der Lampe",
    "Einfach zu warten",
    "Breiter Dimmbereich",
    "Sofort Licht auf der Leinwand dank Heißwiederzündungsfunktion",
    "Kurzbogen mit sehr hoher Leuchtdichte für hellere Leinwandausleuchtung",
]

Make sure to query "application_areas" and "product_family_benefits" with only these values.
Make sure not to filter based on the content and the embedding field. Prefer querying numeric fields based on ecpert advice.

When the query returns products, only return the following fields: product_name, content.

Output the valid sql query only

Only use the following tables:
{table_info}
Also do not grab identifiers and product names from the example data here for composing the query.
Question: {input}"""


answer_prompt_template = """
You are an expert in Xenon short-arc discharge lamps made by OSRAM, typically used in cinema projectors, searchlights, and industrial applications.
Based on question from user, advice from expert, database sql query results, please recommend the appropriate product to the user.

Expert advice: {expert_advice}
Database query result: {db_results}
Question: {input}

Make sure to mention the relevant product specs related to the user question.
Use the data to construct the answer, however Do not mention the database or the expert advice, the sources of data, or products you do not recommend.
Also make sure to recommend the most suitable product.
"""


In [3]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.chains.sql_database.query import create_sql_query_chain
from langchain_community.tools import QuerySQLDatabaseTool
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.globals import set_debug
from app.embedding import XenonLampEmbeddingSystem
from app.models import XenonLamp


from app.populate_db import get_db_params_from_env

set_debug(True)

question = "How many xenon lamps are there?"
db_params = get_db_params_from_env()
db_uri = f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
db = SQLDatabase.from_uri(db_uri)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0, google_api_key=os.environ["GEMINI_API_KEY"])
llm_thinking = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", temperature=0, google_api_key=os.environ["GEMINI_API_KEY"])


questions = [
    "How much does the XBO 4000 W/HS XL OFR weigh?",
    "Which luminaire is best suited for my home theater?",
    "Give me all lamps with at least 1500W and a lifetime of more than 3000 hours.",
    "What is the smallest unit I can order?",
    "Which luminaire has the SCIP number dd2ddf15-037b-4473-8156-97498e721fb3?",
]


embedding_system = XenonLampEmbeddingSystem()
query_embedding = embedding_system.generate_embedding("4500 W/HS")

POSTGRES_PROMPT = PromptTemplate(
    input_variables=["question", "table_info", "top_k", "expert_advice"],
    template=sql_gen_prompt,
)
def parse_sql(query):
    return query.replace("```sql", "").replace("```", "")

def get_expert_advice(inputs):
    expert_prompt = domain_expert_prompt_template.format(json_schema=XenonLamp.model_json_schema(), input=inputs["question"])
    advice = llm.predict(expert_prompt)
    return {**inputs, "expert_advice": advice}

def run_sql_query(inputs):
    sql = parse_sql(inputs["sql"])
    tool = QuerySQLDatabaseTool(db=db)
    db_results = tool.invoke(sql)
    return {**inputs, "db_results": db_results}

def compose_answer(inputs):
    answer_prompt = answer_prompt_template.format(
        input=inputs["question"],
        expert_advice=inputs["expert_advice"],
        db_results=inputs["db_results"]
    )
    answer = llm.predict(answer_prompt)
    print({**inputs, **{"answer": answer}})
    return {"answer": answer}

question_answer_map = {}
chain = (
    RunnableLambda(lambda x: {**x, **get_expert_advice(x)})
    | RunnableLambda(lambda x: {**x, "sql": create_sql_query_chain(llm, db, POSTGRES_PROMPT).invoke(x)})
    | RunnableLambda(run_sql_query)
    | RunnableLambda(compose_answer)
)
for question in questions:
    inputs = {
        "question": question,
    }
    
    result = chain.invoke(inputs)
    question_answer_map[question] = result


/Users/adamkecskes/dev/rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/adamkecskes/dev/rag/.venv/lib/python3.11/site-packages/langchain_community/utilities/sql_database.py:134: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(
Input ids are automatically padded to be a multiple of `config.attention_window`: 512


[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "How much does the XBO 4000 W/HS XL OFR weigh?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableLambda] Entering Chain run with input:
{
  "question": "How much does the XBO 4000 W/HS XL OFR weigh?"
}
[llm/start] [llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an expert in Xenon short-arc discharge lamps made by OSRAM, typically used in cinema projectors, searchlights, and industrial applications.\n Give some hints on how to select the appropriate lighting or lightings for the customer's use case. We have the following information about the lightings: {'properties': {'product_name': {'title': 'Product Name', 'type': 'string'}, 'family_color_temperature_k': {'title': 'Family Color Temperature K', 'type': 'number'}, 'family_power_min_w': {'title': 'Family Power Min W', 'type': 'number'}, 'family_power_max_w': {'title': 'Family Power Max W', 'type': 

/var/folders/t3/nq5nrzjd15gbr_h8k7j48tmc0000gn/T/ipykernel_36619/299308913.py:45: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  advice = llm.predict(expert_prompt)


[llm/end] [llm:ChatGoogleGenerativeAI] [1.15s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To answer the customer's question about the weight of the \"XBO 4000 W/HS XL OFR\" lamp, filter by `product_name` = \"XBO 4000 W/HS XL OFR\" and retrieve the value of `product_weight_g`.",
        "generation_info": {
          "finish_reason": "STOP",
          "model_name": "gemini-2.0-flash",
          "safety_ratings": []
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "To answer the customer's question about the weight of the \"XBO 4000 W/HS XL OFR\" lamp, filter by `product_name` = \"XBO 4000 W/HS XL OFR\" and retrieve the value of `product_weight_g`.",
            "response_metadata": {
              "prompt_feedback